# Image Recognition

In [1]:
import polars as pl
import pandas as pd
import json
import os
import openai
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
import time
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# elements = partition_pdf(filename='nflx-20230930.pdf', infer_table_structure=True)
# Fail!

ERROR
-> 3284 im = _open_core(fp, filename, prefix, formats)
   3286 if im is None and formats is ID:
   3287     checked_formats = formats.copy()

File ~/.genai0/lib/python3.9/site-packages/PIL/Image.py:3271, in open.<locals>._open_core(fp, filename, prefix, formats)
   3269         fp.seek(0)
   3270         im = factory(fp, filename)
-> 3271         _decompression_bomb_check(im.size)
   3272         return im
   3273 except (SyntaxError, IndexError, TypeError, struct.error):
   3274     # Leave disabled by default, spams the logs with image
   3275     # opening failures that are entirely expected.
   3276     # logger.debug("", exc_info=True)

File ~/.genai0/lib/python3.9/site-packages/PIL/Image.py:3179, in _decompression_bomb_check(size)
   3174 if pixels > 2 * MAX_IMAGE_PIXELS:
   3175     msg = (
   3176         f"Image size ({pixels} pixels) exceeds limit of {2 * MAX_IMAGE_PIXELS} "
   3177         "pixels, could be decompression bomb DOS attack."
   3178     )
-> 3179     raise DecompressionBombError(msg)
   3181 if pixels > MAX_IMAGE_PIXELS:
   3182     warnings.warn(
   3183         f"Image size ({pixels} pixels) exceeds limit of {MAX_IMAGE_PIXELS} pixels, "
   3184         "could be decompression bomb DOS attack.",
   3185         DecompressionBombWarning,
   3186     )

DecompressionBombError: Image size (436200720 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.

**El tamañoo del pdf es muy grande y excede el límite que fija la librería**

In [3]:
elements = partition_pdf(filename='netflix_10k-oficialpdf.pdf', infer_table_structure=True)
tables = [el for el in elements if el.category == "Table"]
print(tables[0].text)

Large accelerated filer Xi Non-accelerated filer a


In [4]:
print(tables[0].metadata.text_as_html)

<table><thead><th>Large accelerated filer</th><th>O®</th></thead><tr><td>Non-accelerated filer</td><td></td></tr></table>


In [8]:
text = tables[20].text
html = tables[20].metadata.text_as_html

In [9]:
def llm():
    return AzureChatOpenAI(model_name="gtp35turbo-latest")

model = llm()

message = HumanMessage(
    content= f'You are a financial assistant, convert this html table related to 10-k report in tabular format in simple text: "{html}". Only return the text'
)

html_result = model([message])

In [10]:
print(html_result.content)

Sash flows from operating activities:
Net income $ 4,491,924 $ 5,116,228 $ 2,761,395
Adjustments to reconcile net income to net cash provided by operating activities:
Additions to content assets (16,839,038) (17,702,202) (11,779,284)
Change in content liabilities 179,310 232,898 (757,433)
Amortization of content assets 14,026,132 12,230,367 10,806,912
Depreciation and amortization of property, equipment and intangibles 336,682 208,412 115,710
Stock-based compensation expense 575,452 403,220 415,180
Foreign currency remeasurement loss (gain) on debt (353,111) (430,661) 533,278
Other non-cash items 533,543 376,777 293,126
Deferred income taxes (166,550) 199,548 70,066
Changes in operating assets and liabilities:
Other current assets (353,834) (369,681) (187,623)
Accounts payable (158,543) 145,115 (41,605)
Accrued expenses and other liabilities (55,513) 180,338 198,183
Deferred revenue 27,356 91,350 193,247
Other non-current assets and liabilities (217,553) (289,099) (194,075)
Net cash pr